# Overview
In class today, we learned about loading, manipulating, and visualizing EEG data. For the homework, we will clip out particular sections of EEG data that correspond to experimental events, and create event-related averages, as we did for the fMRI data last week. 

## Questions
1. [3 points] Based on the code we wrote in class, write code to slice out the section of EEG data that corresponds to times from 100 ms (0.1 seconds) *before* the onset of each event and 600 ms (0.6 seconds) *after* the onset of each event for condition 1. Create a 3D array variable from these slices of the data that is of shape (repeats_per_condition, eeg_channels, time). 

2. [3 points] Compute the average response to condition1 across repeats. Then (using the variable created in the previous step) compute the average response across all electrodes. 

3. [2 points] Plot the resulting average over time (see the breakout lecture for an example of how to create a `time` variable to plot against!) Note that the time value on the x axis should range from -0.1 to 0.6! 

4. [3 points] Repeat steps 1-3 for conditions 2-6. Plot the average response for each condition (across repeats and across channels) on the same plot (this will result in a plot of six lines). Label the plot! (Label both axes and create a legend that indicates which line is which - look into the help for `plt.plot()` and `plt.legend()` for how to do this)

## BONUS (extra credit)
1. [2 points] make your plot pretty! Look up how to change the size, colors and widths of lines, etc. 

2. [3 points] If you have done the above steps correctly, you will see a plot of six lines. These lines will not start at the same place at the far left side of the plot, though! This means that during a blank period in the experiment, the response was not the same for all conditions (which we can assume it should be, since the screen was blank at that time). One way to account for this issue is to do the following for each trial (and each electrode): 
    * compute the mean response for the first 0.1 seconds (from -0.1 seconds [100 ms before trial onset] to 0 seconds [trial onset])
    * subtract that mean from the rest the values for that trial, for that electrode

For extra credit, try this, and plot the results!

In [ ]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import neurods
from scipy.stats import zscore
import mne

In [ ]:
%matplotlib inline

In [ ]:
# Function to load eeg data (from lecture, will eventually be in neurods)
def load_eeg_data(fname, ref_electrode_idx=(6,23), verbose=False,
                  filter_cutoffs=(1,12), return_mne=True):
    """Load EEG data"""
    raw = mne.io.Raw(fname, preload=True, verbose=verbose)
    # Remove mastoid channels
    raw._data = raw._data[:-2]
    # Reference signal to temporal lobe electrodes
    if ref_electrode_idx is not None:
        ref_electrodes = raw._data[list(ref_electrode_idx)]
        raw._data -= ref_electrodes.mean(0)
    # Smooth the data
    if filter_cutoffs is not None:
        fmin, fmax = filter_cutoffs # Set cutoffs for low and high frequency in data
        raw._data = mne.filter.filter_data(raw._data, raw.info['sfreq'], fmin, fmax, verbose=verbose)
    if return_mne:
        return raw
    else:
        return raw.times, raw._data

# Load data

In [ ]:
subject, session = 8, 1
fdir = os.path.join(neurods.io.data_list['eeg'], 'subject{}', 'session{}').format(subject, session)
# Load data from file
datafile = os.path.join(fdir, 'eeg-raw.fif')
times, data = load_eeg_data(datafile, return_mne=False)
# Load events from file
eventfile = os.path.join(fdir, 'events-eve.txt')
events = mne.read_events(eventfile)

### Question 1: Select events (epochs) for condition 1
Modify the following code to select the times from 100 ms before stimulus onset to 600 ms after stimulus onset. There are only two lines of code that need changing! 

In [ ]:
# Starting point code from end of lecture (slightly modified):

# Relevant information
sfreq = 2048. # Sampling frequency
tmin = -0.1 # Start for time to select (relative to condition onset)
tmax = 0.6 # Stop for time to select (relative to condition onset)

# Initialize a variable
cond_epochs = []

# Columns in `events` are [onset_indices, <blank>, condition_number]
# Select event indices in condition 1
cond_idx = events[:, 2]==1 # This creates a logical index, as long as `events`, that is True for 
                           # events in condition 1 and False for all other conditions
onsets = events[cond_idx, 0] # This selects all onset indices in condition 1

# Loop over condition onsets to select particular time points
for onset in onsets:
    slice_start = 0 # ?? WHAT SHOULD THIS BE?
    slice_stop = 10 # ?? WHAT SHOULD THIS BE? (10 is NOT correct)
    data_slice = data[:, slice_start:slice_stop]
    data_slice = zscore(data_slice, axis=1)
    cond_epochs.append(data_slice)
cond_epochs = np.array(cond_epochs)
# Dimensions are (repeats, eeg channels, time)
print(cond_epochs.shape)

### Question 2: Compute event-related average for condition 1
(this is a short answer...)

In [ ]:
### STUDENT ANSWER


### Question 3: Plot Condition 1

In [ ]:
### STUDENT ANSWER


### Question 4: Compute the above and plot results for all conditions

In [ ]:
### STUDENT ANSWER


# Bonus!
If you care to try.

In [ ]:
### STUDENT ANSWER
